In [14]:
import pandas as pd

df = pd.read_csv('~/Downloads/GS_1m.csv',sep=";",decimal=',', index_col=None, header=0)
df['timestamp'] = pd.to_datetime(df['timestamp'])

df['date'] = [d.date() for d in df['timestamp']]
df['time'] = [d.time() for d in df['timestamp']]
df['price'] = df['close']

In [70]:
import numpy as np
def volatility(df, intvl=5):
    '''
    
    :param df: stock data frame, columns must have ['date', 'time', 'price'] 
    :param int: time interval, unit in minute
    :return: return a new data frame with its volatility
    '''
    
    data = df.copy()
    output = pd.DataFrame()
    
    for date in set(df.loc[:, 'date']):
        temp_df1 = data.loc[data.date == date, :]
        temp_df1['ind'] = temp_df1.index

        temp_df1 = temp_df1.groupby(temp_df1.index//intvl).mean()
        temp_df2 = temp_df1.set_index('ind')
        temp_df2.loc[:, 'time'] = df.loc[temp_df2.index, 'time']

        s_i = temp_df2['price']
        s_i_1 = temp_df2['price'].shift(1)
        temp_df2['u_sequence'] = np.log(s_i/s_i_1)
        s = temp_df2['u_sequence'].rolling(window=len(set(temp_df1.index//intvl)), center=False).std()
        #data.loc[data.date == date, str(intvl)+'_min_vol'] 
        temp_df2.loc[:, str(intvl)+'_min_vol'] = s * np.sqrt(len(set(temp_df1.index//intvl)))
        
        output = pd.concat([output,temp_df2])
    output.reset_index(drop=True,inplace=True)
    return output


print(volatility(df))

/Users/Maxwell/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


              nr       open       high         low      close         volume  \
0       291918.0  173.20250  173.38500  173.178750  173.34500   84350.000000   
1       291923.0  173.90600  173.93400  173.809000  173.85800   13720.000000   
2       291928.0  173.95402  173.97892  173.922020  173.94492   10760.000000   
3       291933.0  173.97200  173.98000  173.964000  173.97200    7280.000000   
4       291938.0  174.06400  174.08400  174.052000  174.06800    4180.000000   
5       291943.0  174.13000  174.17450  174.122900  174.16640    7580.000000   
6       291948.0  174.41080  174.41880  174.371000  174.38500    9200.000000   
7       291953.0  174.42800  174.46848  174.401000  174.43600    9420.000000   
8       291958.0  174.46600  174.52644  174.446800  174.51044    8460.000000   
9       291963.0  174.46800  174.48000  174.457000  174.46800    4080.000000   
10      291968.0  174.47198  174.49800  174.465980  174.47700    5220.000000   
11      291973.0  174.47600  174.48400  